# Processing the final dataset



In [56]:
import ee
import geemap

# Authenticate to Earth Engine
try:
  ee.Initialize(project='ee-ana-zonia')
except Exception as e:
  ee.Authenticate()
  ee.Initialize(project='ee-ana-zonia')

from utils import export_image
from utils import map_image

In [57]:
# region of interest is Amazonia
roi = ee.FeatureCollection("projects/ee-ana-zonia/assets/br_biomes").first().geometry()
ecoreg = ee.Image("projects/ee-ana-zonia/assets/ecoregions_br")
mask = ee.Image('projects/ee-ana-zonia/assets/one_hectare_secondary_forest_mask').clip(roi)
ecoreg_export = ecoreg.updateMask(mask)

In [58]:
cwd_chave = ee.Image("projects/ee-ana-zonia/assets/cwd_chave")
LU_sum = ee.Image('projects/ee-ana-zonia/assets/LU_sum')
proj = LU_sum.projection()
proj2 = cwd_chave.projection()
cwd_chave2 = cwd_chave.reproject(proj)

In [59]:
LU_sum = ee.Image('projects/ee-ana-zonia/assets/LU_sum')
fallow = ee.Image('projects/ee-ana-zonia/assets/fallow')
last_fire = ee.Image('projects/ee-ana-zonia/assets/last_fire')
num_fires = ee.Image('projects/ee-ana-zonia/assets/num_fires')
frag = ee.Image('projects/ee-ana-zonia/assets/frag_2020')

land_use = LU_sum.addBands([fallow, last_fire, num_fires])

ecoreg = ee.Image("projects/ee-ana-zonia/assets/ecoregions_br")
indig = ee.Image("projects/ee-ana-zonia/assets/indig")
soil = ee.Image("projects/ee-ana-zonia/assets/soil")
protec = ee.Image("projects/ee-ana-zonia/assets/protec")
biomes = ee.Image("projects/ee-ana-zonia/assets/biome_br")
categorical = ecoreg.addBands([indig, soil, protec, biomes])

yearly_si = ee.Image("projects/ee-ana-zonia/assets/yearly_si").float()
mean_prec = ee.Image("projects/ee-ana-zonia/assets/mean_prec").float()
climate = yearly_si.addBands([mean_prec])

biomass = ee.Image('projects/ee-ana-zonia/assets/biomass_masked')
total_export = categorical.addBands([land_use, climate, biomass]).updateMask(mask)

In [61]:
img = total_export
name = "total_export"

# Create the export task
task = ee.batch.Export.image.toAsset(
    image = img,
    description = f'{name}',
    assetId = f'projects/ee-ana-zonia/assets/{name}',
    crs = 'EPSG:4326',
    crsTransform = 
    maxPixels = 4e12,
    region=img.geometry()
)

# Start the export task
task.start()

In [101]:
final = ee.Image('projects/ee-ana-zonia/assets/total_export')

all_band_names = final.bandNames().getInfo()

selected_band_names = [name for name in all_band_names if name.startswith("prec") or name.startswith("si")]

final_float = final.select(selected_band_names).toFloat()

int_band_names = [name for name in all_band_names if not name.startswith("prec") and not name.startswith("si")]
final_int = final.select(int_band_names)

final = final_int.addBands(final_float)

In [102]:
## stratify sample points per feature
stratified = final.stratifiedSample(
    numPoints = 10000,
    classBand = 'ecoreg',
    geometries = True
)

# # Export an ee.FeatureCollection as an Earth Engine asset.
# task = ee.batch.Export.table.toAsset(
#     collection=stratified,
#     description='total_export_w_biomass',
#     assetId='projects/ee-ana-zonia/assets/stratified2',
# )
# task.start()

# Export an ee.FeatureCollection as an Earth Engine asset.
task = ee.batch.Export.table.toDrive(
    collection=stratified,
    description='final_w_biomass'
)
task.start()


In [55]:
total_export_w_soy = total_export.updateMask(LU_sum.select('lulc_sum_39'))

stratified = total_export_w_soy.stratifiedSample(
    numPoints = 10000,
    classBand = 'ecoreg',
    geometries = True
)

# Export an ee.FeatureCollection as an Earth Engine asset.
task = ee.batch.Export.table.toDrive(
    collection=stratified,
    description='total_export_w_biomass_soy'
)
task.start()

